In [30]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, auc

In [26]:
sample_file = 'sample_submission.csv'
label_col = 'Transported'

### Preproc

In [32]:
def read_data():
    # We'll do our own test sets via stratified k-fold later anyway
    train_file = 'train.csv'
    test_file = 'test.csv'
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)
    return df_train, df_test

df_train, df_test = read_data()


In [ ]:
class Preprocessor:
    
    def __init__(self,  
            label_col: List[str], 
            impute_cols: List[str], 
            onehot_cols: List[str], 
            prob_cols: List[str]):
        self.label_col = label_col
        self.impute_cols = impute_cols
        self.onehot_cols = onehot_cols
        self.prob_cols = prob_cols
    
    def fit(self, df):
        
    
    def transform(self, df):
        pass
    

In [29]:
# define available actions
def impute_cols(df, cols):
    for col in cols:
        fill = df[col].mode
        df[col].fillna(fill, inplace=True)

def dropna_cols(df, cols):
    df[col].dropna(subset=cols, inplace=True)

def onehot_cols(df, cols):
    for col in cols:
        

    df.dropna()
    df['Transported'] = df['Transported'].map(int)
    df['VIP'] = df['VIP'].map(int)
    df['CryoSleep'] = df['CryoSleep'].map(int)
    return df

df = read_data()
df.head()

ValueError: cannot convert float NaN to integer

In [22]:
df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000,8693.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791,0.503624
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189,0.500016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000,1.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000,1.000000


In [24]:
df_avg_planet = df[['HomePlanet', 'Transported']].groupby('HomePlanet').mean()
df_count_planet = df[['HomePlanet', 'Transported']].groupby('HomePlanet').count()

print(dict(df_avg_planet['Transported']))
dict(df_count_planet['Transported'])

{'Earth': 0.42394611038678837, 'Europa': 0.65884561238855, 'Mars': 0.5230244457077885}


{'Earth': 4602, 'Europa': 2131, 'Mars': 1759}

In [133]:
df[['HomePlanet', 'Transported']].groupby('HomePlanet').agg(['sum','count'])

Transported      
                   sum count
HomePlanet                  
Earth             1951  4602
Europa            1404  2131
Mars               920  1759

In [123]:
"""
Split PassengerID on _ into group_id, sub_id
probably passengers in the same group had similar fates
make feature: portion in group who were transported, after dropna'ing
and do the same for Cabin actually
"""
df_group_sub = df['PassengerId'].str.split('_', expand=True).rename({0: 'group_id', 1: 'sub_id'}, axis=1).astype(int)

def any_transported_in_group(df_train, group_num):
    pass

In [ ]:
def numberize_scale_dropna(df):

print(len(df_train))
df_train = df_train.dropna(subset=[label_col])
print(len(df_train))
df_train.head()

In [ ]:
df_train['Transported'].unique()

In [87]:
pd.read_csv(sample_file).head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [88]:
y_col = 'Transported'
# x_cols = [col for col in df_train.columns if col != y_col]
x_cols = "RoomService FoodCourt ShoppingMall Spa VRDeck".split()
df_train = df_train[x_cols + [y_col]].dropna()
x_train = df_train[x_cols]
x_train = x_train.dropna()
x_train.head()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,0.0,0.0,0.0,0.0
1,109.0,9.0,25.0,549.0,44.0
2,43.0,3576.0,0.0,6715.0,49.0
3,0.0,1283.0,371.0,3329.0,193.0
4,303.0,70.0,151.0,565.0,2.0


In [89]:
y_train = df_train[y_col].astype(bool)
y_train.head()

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

In [90]:
nbc = GaussianNB()
nbc.fit(x_train, y_train)

GaussianNB()

In [91]:
y_train_hat = nbc.predict(x_train)
confusion_matrix(y_train, y_train_hat)

array([[1551, 2322],
       [ 216, 3696]])